TO DO:
- Add option to force players into the team, i.e., optimize given certain player are already locked in

In [1]:
# previous gameweek
gameweek=34

In [2]:
import pandas as pd
import numpy as np
import json
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)

# Solver definition

In [4]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES

    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    captaincy_variables = [LpVariable(name=f'cap_{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'price'] for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'position']=='GK' else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'position']=='DEF' else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'position']=='MID' else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'position']=='FWD' else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # select 1 captain
    model += lpSum(captaincy_variables) == 1
    # captain must be one of the 11 players in the team
    for i in range(0,len(variables)):
        model += captaincy_variables[i] <= variables[i]
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3

    # OBJECTIVE
    # the second part doubles the captain's points
    model += np.dot(expected_points,variables) + np.dot(expected_points,captaincy_variables)

    # SOLVE OPTIMIZATION

    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [str(var) for var in model.variables() if var.value()==1]
    captain = [player for player in players if 'cap' in player]
    captain = int(captain[0].replace('cap_', ''))
    players = [int(player) for player in players if 'cap' not in player]
    dream_team = df.loc[players]
    cost = (dream_team['price']).sum()
    exp_points = dream_team[expected_column].sum() + df.loc[captain, expected_column]
    print(f'Cost: {cost}')
    print(f'Expected points per week: {exp_points}')
    
    display(dream_team[['position','name', 'team', 'price',expected_column]].sort_values('position'))
    
    return players

# Data

In [5]:
#projections = pd.read_csv(f'../point_projections/xP_projections_week{gameweek}.csv', index_col=0)
projections = pd.read_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv', index_col=0)
projections = projections.rename(columns={'xP':'projected_points'})
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std
0,Granit Xhaka,Arsenal,Newcastle Utd,0,35,2.721090,0.150307
1,Granit Xhaka,Arsenal,Brighton,1,36,2.910805,0.127589
2,Granit Xhaka,Arsenal,Nottingham Forest,0,37,3.551062,0.266156
3,Granit Xhaka,Arsenal,Wolves,1,38,3.740756,0.224557
4,Mohamed Elneny,Arsenal,Newcastle Utd,0,35,2.128134,0.146237


In [6]:
# decay for future projected points to model uncertainty increasing over time
decay_coeff = 0.875
projections['projected_points_with_decay'] = ( projections['projected_points']*decay_coeff
                                              **(projections['gameweek']-(gameweek+1)) )
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std,projected_points_with_decay
0,Granit Xhaka,Arsenal,Newcastle Utd,0,35,2.721090,0.150307,2.721090
1,Granit Xhaka,Arsenal,Brighton,1,36,2.910805,0.127589,2.546955
2,Granit Xhaka,Arsenal,Nottingham Forest,0,37,3.551062,0.266156,2.718782
3,Granit Xhaka,Arsenal,Wolves,1,38,3.740756,0.224557,2.506014
4,Mohamed Elneny,Arsenal,Newcastle Utd,0,35,2.128134,0.146237,2.128134


In [7]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
fpl_df.head(5)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,84450,0,0,-2,2,1,3,3.4,...,2022-23,Granit Xhaka,MID,34,4.8,285182.650,5138,Chelsea,2023-05-02T19:00:00Z,1.0
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,...,2022-23,Mohamed Elneny,MID,34,4.1,68443.836,-211,Chelsea,2023-05-02T19:00:00Z,1.0
2,NaN,NaN,156074,0,0,-3,3,0,2,2.0,...,2022-23,Rob Holding,DEF,34,4.2,22814.612,308,Chelsea,2023-05-02T19:00:00Z,1.0
3,100.0,100.0,167199,0,0,-3,3,0,3,1.4,...,2022-23,Thomas Partey,MID,34,4.7,45629.224,-136,Chelsea,2023-05-02T19:00:00Z,1.0
4,100.0,100.0,184029,0,0,1,-1,5,3,7.2,...,2022-23,Martin degaard,MID,34,6.6,3068565.314,72273,Chelsea,2023-05-02T19:00:00Z,1.0


In [8]:
# choose time horizon for optimization
#horizon = 10

# create dataframe to be given as input for optimization
#df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\
#                                                        [['projected_points','projected_points_with_decay', 'std']] 

# OPTIMIZE FOR GIVEN GAMEWEEKS
my_gameweeks = [35,36,37,38]
df = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).sum()\
                                                        [['projected_points','projected_points_with_decay']]
df['std'] = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).mean()\
                                                        [['std']]
df['number_of_games'] = (projections[projections['gameweek'].isin(my_gameweeks)]
                                     .groupby('name').count()['std'])

#df = df.to_frame()
df = df.reset_index()
#df = df.rename(columns={'projected_points':'projected_points_per_game'})
#df['team'] = [fpl_df.loc[fpl_df['name']==name, 'team'].values[0] for name in df['name']]
df['price'] = [fpl_df.loc[fpl_df['name']==name, 'value'].values[0] for name in df['name']]
df['position'] = [fpl_df.loc[fpl_df['name']==name, 'element_type'].values[0] for name in df['name']]
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['position'].map(position_dict)
df['position'] = pd.Categorical(df['position'], ['GK','DEF','MID','FWD'])
df['team'] = [projections.loc[projections['name']==name, 'team'].head(1).values[0] for name in df['name']]
df['minutes'] = [fpl_df.loc[fpl_df['name']==name, 'minutes'].values[0] for name in df['name']]
# re-order columns
df = df[['name', 'team', 'position', 'price', 'minutes', 'number_of_games', 'projected_points', 'projected_points_with_decay', 'std']]
df.head(5)

/var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ipykernel_13959/1840653496.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).sum()\
/var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ipykernel_13959/1840653496.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['std'] = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).mean()\


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std
0,Aaron Cresswell,West Ham,DEF,4.7,2075,4,11.667507,9.634187,0.119787
1,Aaron Hickey,Brentford,DEF,4.9,1590,4,9.065288,7.527751,0.112022
2,Aaron Ramsdale,Arsenal,GK,4.8,3060,4,13.727799,11.134120,0.136266
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,1076,5,16.429792,13.317192,0.119179
4,Abdoulaye Doucoure,Everton,MID,5.3,1167,4,9.431897,7.679103,0.096335


In [9]:
# remove players who do not have enough minutes played
cutoff = 450
df = df[df['minutes']>=cutoff]

# edit some player prices according to my selling price
df.loc[df['name']=='Erling Haaland', 'price'] = 11.8
df.loc[df['name']=='Martinelli', 'price'] = 6.3

# Optimization

In [10]:
team_value = 102
money_in_the_bank = 0.2
allowed_bench_value = 17.5
max_price = team_value + money_in_the_bank - allowed_bench_value

## Without decay

In [ ]:
players = optimize(df, max_price, 'projected_points_per_game')

In [ ]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Ben Chilwell', 'Riyad Mahrez', 'Darwin Nunez', 'Aymeric Laporte', 'Emil Krafth', 
             'Sergio Gomez', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [ ]:
players = optimize(my_df, max_price, 'projected_points_per_game')

## With decay

In [11]:
#players = optimize(df, max_price, 'projected_points_with_decay')

In [12]:
ban = []

# ban given players
ban_field = ['Aleksandar Mitrovic', 'Aymeric Laporte', 'Riyad Mahrez']
#ban_field = []

ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [13]:
players = optimize(my_df, max_price, 'projected_points_with_decay')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9d6740dba39a442ba562577869752d4a-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9d6740dba39a442ba562577869752d4a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5350 RHS
At line 5747 BOUNDS
At line 6482 ENDATA
Problem MODEL has 396 rows, 734 columns and 2746 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 243.784 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -243.784
Cbc0038I Before mini branch and bound, 734 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,17.541101
137,DEF,Diogo Dalot,Manchester Utd,4.7,15.062933
273,DEF,John Stones,Manchester City,5.6,15.587787
306,DEF,Kieran Trippier,Newcastle Utd,6.1,18.120685
291,MID,Kaoru Mitoma,Brighton,5.6,19.457091
300,MID,Kevin De Bruyne,Manchester City,12.2,22.870146
392,MID,Mohamed Salah,Liverpool,13.0,24.140197
505,MID,Solly March,Brighton,5.3,25.499604
68,MID,Bruno Fernandes,Manchester Utd,9.4,20.611932
165,FWD,Erling Haaland,Manchester City,11.8,20.870985


# Sensitivity analysis

Here we add small random Gaussian noise to each players' projected points and run the optimization with these new estimates. We repeat the process n times and keep a list of how many times each player appeared in the optimal team.

In [14]:
ban = []

# ban given players
ban_field = ['Aleksandar Mitrovic', 'Aymeric Laporte', 'Cristiano Ronaldo']
#ban_field = []

ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [15]:
metric = 'projected_points_with_decay'
# set the standard deviation of the Gaussian noise to be a fraction of each players' projected points
#sigma_factor = 0.1
simulation_rounds = 100
results = []
for i in range(0,simulation_rounds):
    df_with_noise = my_df.copy()
    #df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise[metric]*sigma_factor)
    df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise['std'])*df_with_noise['number_of_games']
    players = optimize(df_with_noise, max_price, metric)
    results.append(players)

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/431b5925380045afb79b74462a6a8451-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/431b5925380045afb79b74462a6a8451-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5349 RHS
At line 5746 BOUNDS
At line 6481 ENDATA
Problem MODEL has 396 rows, 734 columns and 2745 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 237.366 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -237.366
Cbc0038I Before mini branch and bound, 734 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,18.076833
256,DEF,Joao Cancelo,Manchester City,7.1,15.314987
306,DEF,Kieran Trippier,Newcastle Utd,6.1,16.965265
536,DEF,Trent Alexander-Arnold,Liverpool,7.6,16.541320
38,MID,Antony,Manchester Utd,7.3,20.120827
392,MID,Mohamed Salah,Liverpool,13.0,23.314225
433,MID,Pascal Gro,Brighton,5.4,20.110875
462,MID,Riyad Mahrez,Manchester City,7.3,20.175581
505,MID,Solly March,Brighton,5.3,25.844812
19,FWD,Alexander Isak,Newcastle Utd,6.9,17.816366


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/1f53a2d94b6a43dabe18729f8b20fa4a-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/1f53a2d94b6a43dabe18729f8b20fa4a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5349 RHS
At line 5746 BOUNDS
At line 6481 ENDATA
Problem MODEL has 396 rows, 734 columns and 2745 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 232.084 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -232.084
Cbc0038I Before mini branch and bound, 734 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,15.724186
137,DEF,Diogo Dalot,Manchester Utd,4.7,15.580851
306,DEF,Kieran Trippier,Newcastle Utd,6.1,17.019666
536,DEF,Trent Alexander-Arnold,Liverpool,7.6,15.557838
291,MID,Kaoru Mitoma,Brighton,5.6,20.956874
300,MID,Kevin De Bruyne,Manchester City,12.2,21.857535
360,MID,Marcus Rashford,Manchester Utd,7.2,19.726119
392,MID,Mohamed Salah,Liverpool,13.0,20.123019
433,MID,Pascal Gro,Brighton,5.4,20.269788
165,FWD,Erling Haaland,Manchester City,11.8,22.903322


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/90ca3eee3886485dbbf069d51aaf59a6-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/90ca3eee3886485dbbf069d51aaf59a6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5349 RHS
At line 5746 BOUNDS
At line 6481 ENDATA
Problem MODEL has 396 rows, 734 columns and 2745 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 257.949 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.4
Cbc0038I Solution found of -257.936
Cbc0038I Before mini branch and bound, 732 integers at bound fixed and 0 continuous
Cbc0038I Full p

...

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,17.791823
137,DEF,Diogo Dalot,Manchester Utd,4.7,15.437052
273,DEF,John Stones,Manchester City,5.6,16.006661
325,DEF,Lewis Dunk,Brighton,4.8,17.296035
300,MID,Kevin De Bruyne,Manchester City,12.2,21.633220
38,MID,Antony,Manchester Utd,7.3,19.654997
392,MID,Mohamed Salah,Liverpool,13.0,23.801806
505,MID,Solly March,Brighton,5.3,25.472423
71,MID,Bukayo Saka,Arsenal,8.2,20.818702
165,FWD,Erling Haaland,Manchester City,11.8,22.452996


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/5994e1bbf3884704b0e158387ede7a3a-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/5994e1bbf3884704b0e158387ede7a3a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5349 RHS
At line 5746 BOUNDS
At line 6481 ENDATA
Problem MODEL has 396 rows, 734 columns and 2745 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 268.427 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -268.427
Cbc0038I Before mini branch and bound, 734 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,18.933797
306,DEF,Kieran Trippier,Newcastle Utd,6.1,19.761274
325,DEF,Lewis Dunk,Brighton,4.8,17.263972
536,DEF,Trent Alexander-Arnold,Liverpool,7.6,17.399926
215,MID,Jack Grealish,Manchester City,7.3,21.701557
392,MID,Mohamed Salah,Liverpool,13.0,25.672593
462,MID,Riyad Mahrez,Manchester City,7.3,25.459287
505,MID,Solly March,Brighton,5.3,28.458445
68,MID,Bruno Fernandes,Manchester Utd,9.4,25.890188
165,FWD,Erling Haaland,Manchester City,11.8,20.651522


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9fe29bf3d60843fba62c770f4c34d84b-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9fe29bf3d60843fba62c770f4c34d84b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 401 COLUMNS
At line 5349 RHS
At line 5746 BOUNDS
At line 6481 ENDATA
Problem MODEL has 396 rows, 734 columns and 2745 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 249.944 - 0.01 seconds
Cgl0004I processed model has 394 rows, 734 columns (734 integer (734 of which binary)) and 2569 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -249.944
Cbc0038I Before mini branch and bound, 734 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

,position,name,team,price,projected_points_with_decay
246,GK,Jason Steele,Brighton,4.0,18.583616
273,DEF,John Stones,Manchester City,5.6,16.924510
306,DEF,Kieran Trippier,Newcastle Utd,6.1,20.066056
455,DEF,Reece James,Chelsea,5.8,16.046806
291,MID,Kaoru Mitoma,Brighton,5.6,20.617070
300,MID,Kevin De Bruyne,Manchester City,12.2,21.571682
38,MID,Antony,Manchester Utd,7.3,19.167933
505,MID,Solly March,Brighton,5.3,25.755502
68,MID,Bruno Fernandes,Manchester Utd,9.4,21.530493
165,FWD,Erling Haaland,Manchester City,11.8,22.432501


In [16]:
my_df['dream_team_appearances'] = [sum(team.count(player_id) for team in results) for player_id in my_df.index]
my_df.head(5)

,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
0,Aaron Cresswell,West Ham,DEF,4.7,2075,4,11.667507,9.634187,0.119787,0
1,Aaron Hickey,Brentford,DEF,4.9,1590,4,9.065288,7.527751,0.112022,0
2,Aaron Ramsdale,Arsenal,GK,4.8,3060,4,13.727799,11.134120,0.136266,0
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,1076,5,16.429792,13.317192,0.119179,1
4,Abdoulaye Doucoure,Everton,MID,5.3,1167,4,9.431897,7.679103,0.096335,0


In [17]:
print('Most appearances, all players:')
display(my_df.sort_values(by='dream_team_appearances', ascending=False).head(20))

Most appearances, all players:


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
306,Kieran Trippier,Newcastle Utd,DEF,6.1,2892,5,21.642415,18.120685,0.386377,93
505,Solly March,Brighton,MID,5.3,2687,6,30.780140,25.499604,0.672787,92
392,Mohamed Salah,Liverpool,MID,13.0,2933,4,29.429043,24.140197,0.620579,91
246,Jason Steele,Brighton,GK,4.0,810,6,21.199554,17.541101,0.260994,89
19,Alexander Isak,Newcastle Utd,FWD,6.9,1073,5,22.099565,18.522272,0.274301,88
300,Kevin De Bruyne,Manchester City,MID,12.2,2255,5,27.572686,22.870146,0.506925,73
165,Erling Haaland,Manchester City,FWD,11.8,2494,5,24.989663,20.870985,1.037174,67
68,Bruno Fernandes,Manchester Utd,MID,9.4,2879,5,25.324610,20.611932,0.637191,50
137,Diogo Dalot,Manchester Utd,DEF,4.7,2059,5,18.561669,15.062933,0.130940,34
291,Kaoru Mitoma,Brighton,MID,5.6,1838,6,23.509982,19.457091,0.193528,33


In [18]:
print('GOALKEEPERS')
display(my_df[my_df['position']=='GK'].sort_values(by='dream_team_appearances', ascending=False).head(10))

GOALKEEPERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
246,Jason Steele,Brighton,GK,4.0,810,6,21.199554,17.541101,0.260994,89
123,David de Gea,Manchester Utd,GK,5.0,2970,5,17.698173,14.352408,0.146920,5
151,Ederson,Manchester City,GK,5.4,2880,5,18.679598,15.384873,0.180815,3
464,Robert Sanchez,Brighton,GK,4.5,2070,6,17.934061,14.840443,0.175523,2
299,Kepa Arrizabalaga,Chelsea,GK,4.6,2205,5,16.086788,13.290403,0.160726,1
2,Aaron Ramsdale,Arsenal,GK,4.8,3060,4,13.727799,11.134120,0.136266,0
207,Illan Meslier,Leeds United,GK,4.5,3060,4,9.839447,7.994363,0.110156,0
487,Sam Johnstone,Crystal Palace,GK,4.4,450,4,14.616653,11.951342,0.117059,0
418,Nick Pope,Newcastle Utd,GK,5.4,2901,5,15.667183,13.097436,0.153733,0
364,Mark Travers,Bournemouth,GK,4.2,945,4,10.920163,9.006241,0.137411,0


In [19]:
print('DEFENDERS')
display(my_df[my_df['position']=='DEF'].sort_values(by='dream_team_appearances', ascending=False).head(20))

DEFENDERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
306,Kieran Trippier,Newcastle Utd,DEF,6.1,2892,5,21.642415,18.120685,0.386377,93
137,Diogo Dalot,Manchester Utd,DEF,4.7,2059,5,18.561669,15.062933,0.130940,34
273,John Stones,Manchester City,DEF,5.6,1728,5,18.900222,15.587787,0.154343,31
536,Trent Alexander-Arnold,Liverpool,DEF,7.6,2562,4,18.252250,15.011060,0.374679,25
325,Lewis Dunk,Brighton,DEF,4.8,2880,6,18.367485,15.198889,0.237917,22
441,Pervis Estupinan,Brighton,DEF,4.9,2179,6,18.460787,15.280596,0.158784,20
474,Ruben Dias,Manchester City,DEF,6.0,1907,5,18.852246,15.532091,0.186994,20
171,Fabian Schar,Newcastle Utd,DEF,5.1,2757,5,17.744062,14.830348,0.130109,17
269,Joel Veltman,Brighton,DEF,4.6,2050,6,17.403010,14.409052,0.172274,8
381,Maximilian Wober,Leeds United,DEF,4.4,935,4,10.535505,8.568493,1.258049,7


In [20]:
print('MIDFIELDERS')
display(my_df[my_df['position']=='MID'].sort_values(by='dream_team_appearances', ascending=False).head(20))

MIDFIELDERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
505,Solly March,Brighton,MID,5.3,2687,6,30.780140,25.499604,0.672787,92
392,Mohamed Salah,Liverpool,MID,13.0,2933,4,29.429043,24.140197,0.620579,91
300,Kevin De Bruyne,Manchester City,MID,12.2,2255,5,27.572686,22.870146,0.506925,73
68,Bruno Fernandes,Manchester Utd,MID,9.4,2879,5,25.324610,20.611932,0.637191,50
291,Kaoru Mitoma,Brighton,MID,5.6,1838,6,23.509982,19.457091,0.193528,33
462,Riyad Mahrez,Manchester City,MID,7.3,1480,5,22.706461,18.763867,0.578934,32
215,Jack Grealish,Manchester City,MID,7.3,2039,5,22.908458,18.933832,0.392287,30
38,Antony,Manchester Utd,MID,7.3,1527,5,22.666349,18.414925,0.266251,21
71,Bukayo Saka,Arsenal,MID,8.2,2858,4,21.592276,17.594688,0.444807,14
433,Pascal Gro,Brighton,MID,5.4,2722,6,21.980368,18.208422,0.236487,13


In [21]:
print('FORWARDS')
display(my_df[my_df['position']=='FWD'].sort_values(by='dream_team_appearances', ascending=False).head(20))

FORWARDS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
19,Alexander Isak,Newcastle Utd,FWD,6.9,1073,5,22.099565,18.522272,0.274301,88
165,Erling Haaland,Manchester City,FWD,11.8,2494,5,24.989663,20.870985,1.037174,67
190,Harry Kane,Tottenham,FWD,11.4,3046,4,20.842200,17.164465,0.516918,20
74,Callum Wilson,Newcastle Utd,FWD,7.0,1543,5,19.588494,16.399478,0.229053,15
212,Ivan Toney,Brentford,FWD,7.5,2863,4,18.508787,15.434036,0.313990,3
116,Danny Welbeck,Brighton,FWD,6.5,1615,6,19.598228,16.209162,0.182287,3
178,Gabriel Jesus,Arsenal,FWD,8.1,1722,4,18.303221,14.826827,0.282333,2
414,Neal Maupay,Everton,FWD,5.6,1048,4,9.782881,7.963943,0.144049,0
285,Julian Alvarez,Manchester City,FWD,6.0,1059,5,14.182806,11.690049,0.196619,0
287,Kai Havertz,Chelsea,FWD,7.4,2260,5,18.353397,15.157890,0.158438,0
